<img src="https://github.com/luishernand/pandas_fundamentals/blob/master/logo4.JPG?raw=true" height = 200 width=200 alt=" ">  

|Email|Fecha|
|-----|-----|
|luishernandezmatos@yahoo.com|1 de junio 2020|  

---

## Análisis de la cesta de la compra  
Es una técnica habitual utilizada en sectores de Distribución y Gran Consumo (Alimentación, Textil, Perfumería…) y cualquier otro sector que tenga el canal online como uno de los principales canales de distribución (Editorial, Audiovisual…)  

<img src = "https://findialeyva.com/wp-content/uploads/2016/04/An%C3%A1lisis-de-la-cesta-de-la-compra-web.gif" heigth= 200 width = 200 alt=" ">  


El modelo de análisis de la cesta de la compra recorre los datos buscando cómo se relacionan entre ellos. Es decir, trata de encontrar las reglas de asociación que relacionan unos productos con otros en una compra.  

Al analizar un gran volumen de compras de distintos clientes, establece reglas con porcentajes de probabilidad que indican qué artículos se suelen comprar al mismo tiempo, permitiendo conocer más sobre los hábitos de consumo de los clientes habituales.  

El patrón de compra que se encuentra en los datos no solo relaciona pares de productos, si no que también identifica “lotes” de artículos que se suelen vender de forma conjunta aunque a priori no existe una relación directa observable a simple vista.  

Con la información obtenida de un Análisis de la Cesta de la Compra, se pueden poner en marcha acciones como:  

Recolocar los productos en el espacio de venta (físico u online) tratando de acercar aquellos que suelen ir en la misma compra  
Identificar productos “gancho” que consigan aumentar el valor medio de la cesta de la compra
Cross – selling o up – selling, recomendando productos relacionados
Diseñar lotes o paquetes de artículos para potenciar el consumo de uno de ellos
Descatalogar artículos
Acciones con un retorno directo y con impacto en la facturación anual.

### Importar librerías

In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### Cargar [dataset](http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx)

In [30]:
df = pd.read_csv('Online-Retail.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


### Verificar los datos  
1. [x] cantidad de resgitros y columnas
1. [x] Información general
1. [x] Datos faltantes o nulos

In [31]:
df.shape

(541909, 8)

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null float64
Country        541909 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [33]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

### Data Cleanup

Hay una pequeña limpieza, tenemos que hacer:
1. [ ] algunas de las descripciones tienen espacios que deben eliminarse. 
1. [ ] eliminaremos las filas que no tienen números de factura.
1. [ ] Canvertir InvoceNo a datos str
1. [ ] eliminaremos las transacciones de crédito (aquellas con números de factura que contienen C).

In [34]:
# eliminar los espacios de las descripciones
df['Description'] = df['Description'].str.strip()

In [35]:
df.dropna(subset = ['InvoiceNo'], axis=0, inplace = True)

In [36]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [39]:
df = df[~df['InvoiceNo'].str.contains('C')]

### Consolidacin de las transacciónes  
En aras de mantener pequeño el conjunto de datos, solo voy a tomar las ventas de Francia

In [47]:
#filtrar los datos
France = df.query('Country == "France"')

In [62]:
basket = France.pivot_table(index = 'InvoiceNo', columns='Description', values = 'Quantity', aggfunc= sum).fillna(0)
basket.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
# funcion para convertir los datos en 0 y 1
def encode_units(x):
    if x<= 0:
        return 0
    if x >=1:
        return 1
    
basket_set = basket.applymap(encode_units)

In [70]:
basket_set.columns

Index(['10 COLOUR SPACEBOY PEN', '12 COLOURED PARTY BALLOONS',
       '12 EGG HOUSE PAINTED WOOD', '12 MESSAGE CARDS WITH ENVELOPES',
       '12 PENCIL SMALL TUBE WOODLAND', '12 PENCILS SMALL TUBE RED RETROSPOT',
       '12 PENCILS SMALL TUBE SKULL', '12 PENCILS TALL TUBE POSY',
       '12 PENCILS TALL TUBE RED RETROSPOT', '12 PENCILS TALL TUBE WOODLAND',
       ...
       'WRAP VINTAGE PETALS  DESIGN', 'YELLOW COAT RACK PARIS FASHION',
       'YELLOW GIANT GARDEN THERMOMETER', 'YELLOW SHARK HELICOPTER',
       'ZINC  STAR T-LIGHT HOLDER', 'ZINC FOLKART SLEIGH BELLS',
       'ZINC HERB GARDEN CONTAINER', 'ZINC METAL HEART DECORATION',
       'ZINC T-LIGHT HOLDER STAR LARGE', 'ZINC T-LIGHT HOLDER STARS SMALL'],
      dtype='object', name='Description', length=1563)

In [75]:
#Eliminar la columna de POSTAGE (dado que ese cargo no es uno que deseamos explorar):
basket_set.drop('POSTAGE',axis = 1, inplace =True)

### Frequent Items set

generaremos conjuntos de elementos frecuentes que tengan  un soporte de al menos 7% (este número se eligió para poder obtener suficientes ejemplos útiles):

In [76]:
conjunto_frequentes = apriori(basket_set, min_support=0.07, use_colnames=True)

###  reglas con su correspondiente apoyo, confianza y elevación: 

In [77]:
reglas = association_rules(conjunto_frequentes, metric='lift', min_threshold=1)

**El soporte o support** es la frecuencia relativa que muestran las reglas. En muchos casos, es posible que desee buscar un alto soporte para asegurarse de que sea una relación útil. Sin embargo, puede haber casos en los que un bajo soporte sea útil si está tratando de encontrar relaciones "ocultas".  

**La confianza confidence** es una medida de la fiabilidad de la regla. Una confianza de .5 en el ejemplo anterior significaría que en el 50% de los casos en que se compraron pañales y chicles, la compra también incluía cerveza y papas fritas. Para la recomendación del producto, una confianza del 50% puede ser perfectamente aceptable, pero en una situación médica, este nivel puede no ser lo suficientemente alto.

**La elevación lift** es la relación entre el soporte observado y el esperado si las dos reglas fueran independientes (ver [wikipedia](https://en.wikipedia.org/wiki/Association_rule_learning)). La regla básica es que un valor de elevación cercano a 1 significa que las reglas eran completamente independientes. Los valores de elevación> 1 son generalmente más "interesantes" y podrían ser indicativos de un patrón de regla útil.


In [78]:
reglas.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
1,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


---  

Ahora, la parte difícil es descubrir lo que esto nos dice. Por ejemplo, podemos ver que hay bastantes reglas con un alto valor de elevación, lo que significa que ocurre con más frecuencia de lo esperado dada la cantidad de transacciones y combinaciones de productos. También podemos ver varios donde la confianza es alta también. Esta parte del análisis es donde el conocimiento del dominio será útil. Solo buscaré un par de ejemplos ilustrativos.  

Podemos filtrar el marco de datos utilizando el código estándar de pandas. En este caso, busque un elevador grande (6) y una alta confianza  

---

In [82]:
reglas[(reglas.lift >= 6) & (reglas.confidence >= 0.8)  ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
16,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.132653,0.102041,0.800000,6.030769,0.085121,4.336735
18,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.137755,0.122449,0.960000,6.968889,0.104878,21.556122
19,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.127551,0.122449,0.888889,6.968889,0.104878,7.852041
20,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850
21,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796
22,"(SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959


Al observar las reglas, parece que los ***despertadores verde y rojo*** se compran juntos y ***los vasos de papel rojo, las servilletas y los platos*** se compran juntos de una manera que es más alta de lo que sugeriría la probabilidad general.  

En este punto, es posible que desee ver cuántas oportunidades hay para utilizar la popularidad de un producto para impulsar las ventas de otro. 

In [88]:
print('verdes =' + str(basket['ALARM CLOCK BAKELIKE GREEN'].sum()))
print(' Rojos=' + str(basket['ALARM CLOCK BAKELIKE RED'].sum()))

verdes =340.0
 Rojos=316.0


Por ejemplo, podemos ver que vendemos 340 relojes de alarma verde pero solo 316 relojes de alarma rojos, ¿entonces tal vez podamos impulsar más ventas de relojes de alarma rojos a través de recomendaciones?

---

Lo que también es interesante es ver cómo las combinaciones varían según el país de compra. Veamos cuáles podrían ser algunas combinaciones populares en Alemania:

In [89]:
#filtrar los datos
Germany = df.query('Country == "Germany"')

In [91]:
Germany.shape

(9042, 8)

In [92]:
basket_2 = Germany.pivot_table(index = 'InvoiceNo', columns='Description', values = 'Quantity', aggfunc= sum).fillna(0)
basket_2.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE SKULLS,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [93]:
basket_set_germany = basket_2.applymap(encode_units)
basket_set_germany.drop('POSTAGE',axis = 1, inplace =True)

In [94]:
frequent_itemset = apriori(basket_set_germany, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemset, metric='lift', min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.137856,0.115974,0.067834,0.492063,4.242887,0.051846,1.740427
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
2,(PLASTERS IN TIN CIRCUS PARADE),(ROUND SNACK BOXES SET OF 4 FRUITS),0.115974,0.157549,0.050328,0.433962,2.754455,0.032057,1.488330
3,(ROUND SNACK BOXES SET OF 4 FRUITS),(PLASTERS IN TIN CIRCUS PARADE),0.157549,0.115974,0.050328,0.319444,2.754455,0.032057,1.298977
4,(ROUND SNACK BOXES SET OF4 WOODLAND),(PLASTERS IN TIN CIRCUS PARADE),0.245077,0.115974,0.056893,0.232143,2.001685,0.028470,1.151290


In [95]:
rules[(rules.lift>=4) & (rules.confidence>0.05)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.137856,0.115974,0.067834,0.492063,4.242887,0.051846,1.740427
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
6,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
7,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.137856,0.107221,0.061269,0.444444,4.145125,0.046488,1.607002
10,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
11,(WOODLAND CHARLOTTE BAG),(RED RETROSPOT CHARLOTTE BAG),0.126915,0.070022,0.059081,0.465517,6.648168,0.050194,1.739959


In [102]:
print('producto antecedente:' , basket_set_germany['PLASTERS IN TIN WOODLAND ANIMALS'].sum())
print('producto consecuente:' , basket_set_germany['PLASTERS IN TIN CIRCUS PARADE'].sum())

producto antecedente: 63
producto consecuente: 53


# Conclusión  
El aspecto realmente agradable del análisis de asociación es que es fácil de ejecutar y relativamente fácil de interpretar. Si no tuviera acceso a MLxtend y este análisis de asociación, sería extremadamente difícil encontrar estos patrones utilizando el análisis básico de Excel. Con Python y MLxtend, el proceso de análisis es relativamente sencillo y, dado que está en Python, tiene acceso a todas las técnicas de visualización y herramientas de análisis de datos adicionales en el ecosistema de Python.